# Connectionist Temporal Classification Loss(CTC loss)

## 问题
在语音识别、文字识别领域，输入的数据通常是序列数据，并且我们不知道那一段语音、像素区域对应于 label 中的字符，
这个对应过程当然可以用人工的方式来做，但是这种做法费时费力，实际场景中是很低效的。

![](https://distill.pub/2017/ctc/assets/handwriting_recognition.svg)
![](https://distill.pub/2017/ctc/assets/speech_recognition.svg)

对于上述的情况，我们可以使用 Connectionist Temporal Classification (CTC) 来实现原始输入数据和 label 的对齐。

定义输入数据为 $\bf{X}=[x_1, x_2, ..., x_T]$，输出为 $\bf{Y}=[y_1, y_2, ..., y_U]$，两者有以下关系：
- $\bf X$ 和 $\bg Y$ 的长度不一定相同
- $\bf X$ 和 $\bg Y$ 的长度比值不同
- $\bf X$ 和 $\bg Y$ 两者的对应关系未知

CTC 可以克服上面这些问题，最终给出 $\bf X$ 中每一个元素对应于 $\bf Y$ 中各元素的概率。

**Loss Function:** 对于给定的输入，我们要训练我们的模型，使得输出等于 $Y$ 的概率最大，即求解条件概率函数 $p(Y | X)$ 的最大值。

**Inference:** 模型训练好后，给定一个新的输入 $X$，我们可以接如下公式来求输出：

$$Y^*=argmax p(Y|X)$$

$$Y^* = \underset{Y}{\mathrm{argmax}}\ p(\Y|X)$$

## 算法
先以一个简单的例子为例，输入为一个 6 维的向量，label 为 $Y=[c, a, t]$，假设用对简单的对其方法：一个输入值对应一个字符，然后去掉重复的字符作为最终输出。

![](https://distill.pub/2017/ctc/assets/naive_alignment.svg)

这种方法有两个问题：
- 实际情况中并不是所有输入值都应该对应一个输出的，比如语音识别中可能有不说话的部分，OCR 一块像素中可能是背景
- 对于 `hello` 这个单词，永远无法得到正确的结果，去重后只会留下 `helo`

为了解决上述的问题，CTC 引入了一个 `blank` 符号，这里用 $ϵ$ 表示。这个符号代表输入中的值不与字符集中的任何一个元素对应，在最后解码时会被移除，下图解释了加入 `blank` 符后的对齐过程：

1. 合并连续的相同符号
2. 去掉 `blank` 字符。

![](https://distill.pub/2017/ctc/assets/ctc_alignment_steps.svg)

## Loss Function
CTC 通常和 RNN 一起使用，RNN 每一个时间步的输出用来计算每一个 label 的概率 $p_t(a_t|\bf X)$ 。但其实 CTC 也可以和其他结构一起使用，比如 CNN 的输出层接一个全链接层把数据 reshape 成 `[batch_size, time_step, ...]`，也可以把这个数据直接给 CTC。

CTC 对齐的整个过程如下：

![](https://distill.pub/2017/ctc/assets/full_collapse_from_audio.svg)

图中的第三步，需要计算不同输出序列的概率，用暴力的方法求，效率会很低，用**动态规划**来求会快很多。下图是应用动态规划算法的一个示例图，期望的输出为 $Y=[a, b]$，所以有效的起始节点只有两个($ϵ$ 和 $a$)，有效的结束节点也只有两个($ϵ$ 和 $b$)。

![](https://distill.pub/2017/ctc/assets/ctc_cost.svg)

对于每一个有效路径可以计算出它的概率：
$$p(a|X) = \prod_{t=1}^T p(a_t|X)$$

所以对于训练集 $D$，可以通过最小化 negative log-likelihood 方程来求解模型的参数：

$$\sum_{(X, Y) \in D} -\log p(Y|X)$$

In [8]:
"""
使用 softmax 对每一个时间步的值分类，一输入对应 classes 中的一输出
"""
import numpy as np

np.random.seed(1111)

time_steps = 12
vals_dim = 6 # data dim for each time step
num_classes = 5


x = np.random.random([time_steps, vals_dim])
w = np.random.random([vals_dim, num_classes])

"""
使用 softmax 计算每一个时间步输出值的分类概率
"""
def softmax(logits):
    max_value = np.max(logits, axis=1, keepdims=True)
    exp = np.exp(logits - max_value)
    exp_sum = np.sum(exp, axis=1, keepdims=True)
    dist = exp / exp_sum
    return dist

def toy_nw(x):
    print("Input data shape: {}".format(x.shape))
    y = np.matmul(x, w)
    print("Output data shape: {}".format(y.shape))
    y = softmax(y)
    return y

y = toy_nw(x)
print(y)
raw_rs = np.argmax(y, axis=1)

Input data shape: (12, 6)
Output data shape: (12, 5)
[[0.24654511 0.18837589 0.16937668 0.16757465 0.22812766]
 [0.25443629 0.14992236 0.22945293 0.17240658 0.19378184]
 [0.24134404 0.17179604 0.23572466 0.12994237 0.22119288]
 [0.27216255 0.13054313 0.2679252  0.14184499 0.18752413]
 [0.32558002 0.13485564 0.25228604 0.09743785 0.18984045]
 [0.23855586 0.14800386 0.23100255 0.17158135 0.21085638]
 [0.38534786 0.11524603 0.18220093 0.14617864 0.17102655]
 [0.21867406 0.18511892 0.21305488 0.16472572 0.21842642]
 [0.29856607 0.13646801 0.27196606 0.11562552 0.17737434]
 [0.242347   0.14102063 0.21716951 0.2355229  0.16393996]
 [0.26597326 0.10009752 0.23362892 0.24560198 0.15469832]
 [0.23337289 0.11918746 0.28540761 0.20197928 0.16005275]]
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


## 1.3 似然计算
和大多数有监督学习一样，CTC 使用最大似然标准进行训练。

给定输入 $x$，输出 $l$ 的条件概率为：
$$
p(l|x) =  \sum_{\pi \in B^{-1}(l)} p(\pi|x)
$$

其中，$B^{-1}(l)$ 表示了长度为 $T$ 且示经过 $B$ 结果为 $l$ 字符串的集合。

**CTC 假设输出的概率是（相对于输入）条件独立的**，因此有：
$$p(\pi|x) = \prod y^t_{\pi_t}, \forall \pi \in L^{\prime T}$$


然而，直接按上式我们没有办理有效的计算似然值。下面用动态规划解决似然的计算及梯度计算, 涉及前向算法和后向算法。

## 1.4 前向算法

在前向及后向计算中，CTC 需要将输出字符串进行扩展。具体的，$(a_1,\cdots,a_m)$ 每个字符之间及首尾分别插入 blank，即扩展为 $(-, a_1,-,a_2, -,\cdots,-, a_m,-)$。下面的 $l$ 为原始字符串，$l^\prime$ 指为扩展后的字符串。

定义
$$
\alpha_t(s) \stackrel{def}{=} \sum_{\pi \in N^T: B(\pi_{1:t}) = l_{1:s}} \prod_{t^\prime=1}^t y^t_{\pi^\prime}
$$

显然有，

$$
\begin{align}
\alpha_1(1) = y_b^1,\\
\alpha_1(2) = y_{l_1}^1,\\
\alpha_1(s) = 0, \forall s > 2
\end{align}
$$
根据 $\alpha$ 的定义，有如下递归关系：
$$
\alpha_t(s) = \{  \begin{array}{l}
(\alpha_{t-1}(s)+\alpha_{t-1}(s-1)) y^t_{l^\prime_s},\  \  \    if\  l^\prime_s = b \ or\  l_{s-2}^\prime = l_s^{\prime}  \\
(\alpha_{t-1}(s)+\alpha_{t-1}(s-1) + \alpha_{t-1}(s-2)) y^t_{l^\prime_s} \ \ otherwise
\end{array}
$$

### 1.4.1 Case 2
递归公式中 case 2 是一般的情形。如图所示，$t$ 时刻字符为 $s$ 为 blank 时，它可能由于两种情况扩展而来：
1. 重复上一字符，即上个字符也是 a
2. 字符发生转换，即上个字符是非 a 的字符，这种情况又分为两种情况
  1. 上一字符是 blank
  2. a 由非 blank 字符直接跳转而来（$B$） 操作中， blank 最终会被去掉，因此 blank 并不是必须的）。
  
![](https://distill.pub/2017/ctc/assets/cost_regular.svg)
**图2. 前向算法 Case 2 示例【[src](https://distill.pub/2017/ctc/)】**

### 1.4.2 Case 1
递归公式 case 1 是特殊的情形。
如图所示，$t$ 时刻字符为 $s$ 为 blank 时，它只能由于两种情况扩展而来：
1. 重复上一字符，即上个字符也是 blank
2. 字符发生转换，即上个字符是非 blank 字符。$t$ 时刻字符为 $s$ 为非 blank 时，类似于 case 2，但是这时两个相同字符之间的 blank 不能省略（否则无法区分"aa"和"a"），因此，也只有两种跳转情况。

![](https://distill.pub/2017/ctc/assets/cost_no_skip.svg)
**图3. 前向算法 Case 1 【[src](https://distill.pub/2017/ctc/)】**

我们可以利用动态规划计算所有 $\alpha$ 的值，算法时间和空间复杂度为 $O(T * L)$。

似然的计算只涉及乘加运算，因此，CTC 的似然是可导的，可以尝试 tensorflow 或 pytorch 等具有自动求导功能的工具自动进行梯度计算。下面介绍如何手动高效的计算梯度。

In [2]:
def forward(y, labels):
    T, V = y.shape
    L = len(labels)
    alpha = np.zeros([T, L])

    # init
    alpha[0, 0] = y[0, labels[0]]
    alpha[0, 1] = y[0, labels[1]]

    for t in range(1, T):
        for i in range(L):
            s = labels[i]
            
            a = alpha[t - 1, i] 
            if i - 1 >= 0:
                a += alpha[t - 1, i - 1]
            if i - 2 >= 0 and s != 0 and s != labels[i - 2]:
                a += alpha[t - 1, i - 2]
                
            alpha[t, i] = a * y[t, s]
            
    return alpha

labels = [0, 3, 0, 3, 0, 4, 0]  # 0 for blank
alpha = forward(y, labels)
print(alpha)

[[  2.46545113e-01   1.67574654e-01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.27300235e-02   7.13969720e-02   4.26370730e-02   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.51395174e-02   1.74287803e-02   2.75214373e-02   5.54036251e-03
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.12040964e-03   4.61964998e-03   1.22337658e-02   4.68965079e-03
    1.50787918e-03   1.03895167e-03   0.00000000e+00]
 [  1.34152305e-03   8.51612635e-04   5.48713543e-03   1.64898136e-03
    2.01779193e-03   1.37377693e-03   3.38261905e-04]
 [  3.20028190e-04   3.76301179e-04   1.51214552e-03   1.22442454e-03
    8.74730268e-04   1.06283215e-03   4.08416903e-04]
 [  1.23322177e-04   1.01788478e-04   7.27708889e-04   4.00028082e-04
    8.08904808e-04   5.40783712e-04   5.66942671e-04]
 [  2.69673617e-05   3.70815141e-05   1.81389560e-04   1.85767281e-04
    2.64362267e-04   3.82184328e-04   2.42231029e-04]
 [  8.05

最后可以得到似然 $p(l|x) = \alpha_T(|l^\prime|) + \alpha_T(|l^\prime|-1)$。

In [3]:
p = alpha[-1, labels[-1]] + alpha[-1, labels[-2]]
print(p)

6.81811271177e-06


# 3. 解码
训练的 $N_w$ 可以用来预测新的样本输入对应的输出字符串，这涉及到解码。
按照最大似然准则，最优的解码结果为：
$$
h(x) = \underset{l \in L^{\le T}}{\mathrm{argmax}}\ p(l|x)
$$

然而，上式不存在已知的高效解法。下面介绍几种实用的近似解码方法。

## 3.1 贪心搜索 （greedy search）
虽然 $p(l|x)$ 难以有效的计算，但是由于 CTC 的独立性假设，对于某个具体的字符串 $\pi$（去 blank 前），确容易计算：
$$
p(\pi|x) = \prod_{k=1}^T p(\pi_k|x)
$$

因此，我们放弃寻找使 $p(l|x)$ 最大的字符串，退而寻找一个使 $p(\pi|x)$ 最大的字符串，即：

$$
h(x) \approx B(\pi^\star)
$$
其中，
$$
\pi^\star = \underset{\pi \in N^T}{\mathrm{argmax}}\ p(\pi|x)
$$

简化后，解码过程（构造 $\pi^\star$）变得非常简单（基于独立性假设）： 在每个时刻输出概率最大的字符:
$$
\pi^\star = cat_{t=1}^T(\underset{s \in L^\prime}{\mathrm{argmax}}\ y^t_s)
$$




In [12]:
def remove_blank(labels, blank=0):
    new_labels = []
    
    # combine duplicate
    previous = None
    for l in labels:
        if l != previous:
            new_labels.append(l)
            previous = l
            
    # remove blank     
    new_labels = [l for l in new_labels if l != blank]
    
    return new_labels

def insert_blank(labels, blank=0):
    new_labels = [blank]
    for l in labels:
        new_labels += [l, blank]
    return new_labels

def greedy_decode(y, blank=0):
    raw_rs = np.argmax(y, axis=1)
    rs = remove_blank(raw_rs, blank)
    return raw_rs, rs

np.random.seed(1111)
y = softmax(np.random.random([20, 6]))
print("Input shape: {}".format(y.shape))
rr, rs = greedy_decode(y)
print(rr)
print(rs)

Input shape: (20, 6)
[1 3 5 5 5 5 1 5 3 4 4 3 0 4 5 0 3 1 3 3]
[1, 3, 5, 1, 5, 3, 4, 3, 4, 5, 3, 1, 3]


## 3.2 束搜索（Beam Search）
显然，贪心搜索的性能非常受限。例如，它不能给出除最优路径之外的其他其优路径。很多时候，如果我们能拿到 nbest 的路径，后续可以利用其他信息来进一步优化搜索的结果。束搜索能近似找出 top 最优的若干条路径。

In [14]:
def beam_decode(y, beam_size=10):
    T, V = y.shape
    log_y = np.log(y)
    
    beam = [([], 0)]
    for t in range(T):  # for every timestep
        new_beam = []
        for prefix, score in beam:
            for i in range(V):  # for every state
                new_prefix = prefix + [i]
                new_score = score + log_y[t, i]
                
                new_beam.append((new_prefix, new_score))
                
        # top beam_size
        new_beam.sort(key=lambda x: x[1], reverse=True)
        beam = new_beam[:beam_size]
        
    return beam
    
np.random.seed(1111)
y = softmax(np.random.random([20, 6]))
beam = beam_decode(y, beam_size=100)
for string, score in beam[:10]:
    print(remove_blank(string), score)

[1, 3, 5, 1, 5, 3, 4, 3, 4, 5, 3, 1, 3] -29.261797539205567
[1, 3, 5, 1, 5, 3, 4, 3, 3, 5, 3, 1, 3] -29.279020152518033
[1, 3, 5, 1, 5, 3, 4, 2, 3, 4, 5, 3, 1, 3] -29.300726142201842
[1, 5, 1, 5, 3, 4, 3, 4, 5, 3, 1, 3] -29.310307014773972
[1, 3, 5, 1, 5, 3, 4, 2, 3, 3, 5, 3, 1, 3] -29.31794875551431
[1, 5, 1, 5, 3, 4, 3, 3, 5, 3, 1, 3] -29.327529628086438
[1, 3, 5, 1, 5, 4, 3, 4, 5, 3, 1, 3] -29.331572723457334
[1, 3, 5, 5, 1, 5, 3, 4, 3, 4, 5, 3, 1, 3] -29.33263180992451
[1, 3, 5, 4, 1, 5, 3, 4, 3, 4, 5, 3, 1, 3] -29.334649090836038
[1, 3, 5, 1, 5, 3, 4, 3, 4, 5, 3, 1, 3] -29.33969505198154


## 3.3 前缀束搜索（Prefix Beam Search）
直接的束搜索的一个问题是，在保存的 top N 条路径中，可能存在多条实际上是同一结果（经过去重复、去 blank 操作）。这减少了搜索结果的多样性。下面介绍的前缀搜索方法，在搜索过程中不断的合并相同的前缀[2]。参考 [gist](https://gist.github.com/awni/56369a90d03953e370f3964c826ed4b0)，前缀束搜索实现如下：

In [19]:
from collections import defaultdict

def prefix_beam_decode(y, beam_size=10, blank=0):
    T, V = y.shape
    log_y = np.log(y)
    
    beam = [(tuple(), (0, ninf))]  # blank, non-blank
    for t in range(T):  # for every timestep
        new_beam = defaultdict(lambda : (ninf, ninf))
             
        for prefix, (p_b, p_nb) in beam:
            for i in range(V):  # for every state
                p = log_y[t, i]
                
                if i == blank:  # propose a blank
                    new_p_b, new_p_nb = new_beam[prefix]
                    new_p_b = logsumexp(new_p_b, p_b + p, p_nb + p)
                    new_beam[prefix] = (new_p_b, new_p_nb)
                    continue
                else:  # extend with non-blank
                    end_t = prefix[-1] if prefix else None
                    
                    # exntend current prefix
                    new_prefix = prefix + (i,)
                    new_p_b, new_p_nb = new_beam[new_prefix]
                    if i != end_t:
                        new_p_nb = logsumexp(new_p_nb, p_b + p, p_nb + p)
                    else:
                        new_p_nb = logsumexp(new_p_nb, p_b + p)
                    new_beam[new_prefix] = (new_p_b, new_p_nb)
                    
                    # keep current prefix
                    if i == end_t:
                        new_p_b, new_p_nb = new_beam[prefix]
                        new_p_nb = logsumexp(new_p_nb, p_nb + p)
                        new_beam[prefix] = (new_p_b, new_p_nb)
                
        # top beam_size
        beam = sorted(new_beam.items(), key=lambda x : logsumexp(*x[1]), reverse=True)
        beam = beam[:beam_size]
        
    return beam

np.random.seed(1111)
y = softmax(np.random.random([20, 6]))
beam = prefix_beam_decode(y, beam_size=100)
for string, score in beam[:20]:
    print(remove_blank(string), score)

([1, 5, 4, 1, 3, 4, 5, 2, 3], (-18.189863809114193, -17.613677981426175))
([1, 5, 4, 5, 3, 4, 5, 2, 3], (-18.19636512622969, -17.621013424585406))
([1, 5, 4, 1, 3, 4, 5, 1, 3], (-18.317018960331531, -17.666629973270073))
([1, 5, 4, 5, 3, 4, 5, 1, 3], (-18.323388267369936, -17.674125139073176))
([1, 5, 4, 1, 3, 4, 3, 2, 3], (-18.415808498759556, -17.862744326248826))
([1, 5, 4, 1, 3, 4, 3, 5, 3], (-18.366422766638632, -17.898463479112884))
([1, 5, 4, 5, 3, 4, 3, 2, 3], (-18.42224294936932, -17.870025672291458))
([1, 5, 4, 5, 3, 4, 3, 5, 3], (-18.372199113900191, -17.905130493229173))
([1, 5, 4, 1, 3, 4, 5, 4, 3], (-18.457066311773847, -17.880630315602037))
([1, 5, 4, 5, 3, 4, 5, 4, 3], (-18.462614293487096, -17.88759583852546))
([1, 5, 4, 1, 3, 4, 5, 3, 2], (-18.458941701567706, -17.951422824358747))
([1, 5, 4, 5, 3, 4, 5, 3, 2], (-18.464527031120184, -17.958629487208658))
([1, 5, 4, 1, 3, 4, 3, 1, 3], (-18.540857550725587, -17.920589910093689))
([1, 5, 4, 5, 3, 4, 3, 1, 3], (-18.547146

## 3.4 其他解码方法
上述介绍了基本解码方法。实际中，搜索过程可以接合额外的信息，提高搜索的准确度（例如在语音识别任务中，加入语言模型得分信息, [前缀搜索+语言模型](https://github.com/PaddlePaddle/DeepSpeech/blob/develop/decoders/decoders_deprecated.py
)）。

本质上，CTC 只是一个训练准则。训练完成后，$N_w$ 输出一系列概率分布，这点和常规基于交叉熵准则训练的模型完全一致。因此，特定应用领域常规的解码也可以经过一定修改用来 CTC 的解码。例如在语音识别任务中，利用 CTC 训练的声学模型可以无缝融入原来的 WFST 的解码器中[5]（e.g. 参见 [EESEN](https://github.com/srvk/eesen)）。

此外，[1] 给出了一种利用 CTC 顶峰特点的启发式搜索方法。

# 4. 工具

[warp-ctc](https://github.com/baidu-research/warp-ctc) 是百度开源的基于 CPU 和 GPU 的高效并行实现。warp-ctc 自身提供 C 语言接口，对于流利的机器学习工具（[torch](https://github.com/baidu-research/warp-ctc/tree/master/torch_binding)、 [pytorch](https://github.com/SeanNaren/deepspeech.pytorch) 和 [tensorflow](https://github.com/baidu-research/warp-ctc/tree/master/tensorflow_binding)、[chainer](https://github.com/jheymann85/chainer_ctc)）都有相应的接口绑定。

[cudnn 7](https://developer.nvidia.com/cudnn) 以后开始提供 CTC 支持。

Tensorflow 也原生支持 [CTC loss](https://www.tensorflow.org/api_docs/python/tf/nn/ctc_loss)，及 greedy 和 beam search 解码器。

# 小结
1. CTC 可以建模无对齐信息的多对多序列问题（输入长度不小于输出），如语音识别、连续字符识别 [3,4]。
2. CTC 不需要输入与输出的对齐信息，可以实现端到端的训练。
3. CTC 在 loss 的计算上，利用了整个 labels 序列的全局信息，某种意义上相对逐帧计算损失的方法，"更加区分性"。

# References
1. Graves et al. [Connectionist Temporal Classification : Labelling Unsegmented Sequence Data with Recurrent Neural Networks](ftp://ftp.idsia.ch/pub/juergen/icml2006.pdf).
2. Hannun et al. [First-Pass Large Vocabulary Continuous Speech Recognition using Bi-Directional Recurrent DNNs](https://arxiv.org/abs/1408.2873).
3. Graves et al. [Towards End-To-End Speech Recognition with Recurrent Neural Networks](http://jmlr.org/proceedings/papers/v32/graves14.pdf).
4. Liwicki et al. [A novel approach to on-line handwriting recognition based on bidirectional long short-term memory networks](https://www.cs.toronto.edu/~graves/icdar_2007.pdf).
5. Zenkel et al. [Comparison of Decoding Strategies for CTC Acoustic Models](https://arxiv.org/abs/1708.004469).
5. Huannun. [Sequence Modeling with CTC](https://distill.pub/2017/ctc/).

参考：
- https://distill.pub/2017/ctc/
- https://medium.com/corti-ai/ctc-networks-and-language-models-prefix-beam-search-explained-c11d1ee23306
- https://github.com/DingKe/ml-tutorial/blob/master/ctc/CTC.ipynb
- http://hongbomin.com/2017/06/23/beam-search/